In [290]:
import requests
import json
import time
from utils import KISOpenAPI

class StockTrader:
    def __init__(self):
        # 사용자 입력을 통해 config_path 결정
        config_choice = input("1: 운영계, 2: 개발계, 3: 모의투자 계정 파일을 선택하세요: ")

        if config_choice == '1':
            self.config_path = 'config.yaml'  # 운영계 파일
            self.token_file = 'token.json'  # 운영계 token 파일
            self.tr_id_prefix = "T"  # 운영계 tr_id 접두어
        elif config_choice == '2':
            self.config_path = 'config_dev.yaml'  # 개발계 파일
            self.token_file = 'token_dev.json'  # 개발계 token 파일
            self.tr_id_prefix = "T"  # 개발계 tr_id 접두어
        elif config_choice == '3':
            self.config_path = 'config_vts.yaml'  # 모의투자 파일
            self.token_file = 'token_vts.json'  # 모의투자 token 파일
            self.tr_id_prefix = "V"  # 모의투자 tr_id 접두어
        else:
            print("잘못된 입력입니다. 기본값인 'config_dev.yaml'로 설정됩니다.")
            self.config_path = 'config_dev.yaml'
            self.token_file = 'token_dev.json'  # 기본값: 개발계 token 파일
            self.tr_id_prefix = "T"  # 기본값: 개발계

        self.api = KISOpenAPI(config_path=self.config_path, token_file=self.token_file)
        self.ACCESS_TOKEN = self.api.get_token()
        self.URL_BASE = self.api.URL_BASE
        self.CANO = self.api.CANO
        self.ACNT_PRDT_CD = self.api.ACNT_PRDT_CD

    def get_possible_buy_qty(self, code="005930"):
        # 매수 가능 수량 조회
        PATH = "uapi/domestic-stock/v1/trading/inquire-psbl-order"
        URL = f"{self.URL_BASE}/{PATH}"
        headers = {
            "Content-Type": "application/json",
            "authorization": f"Bearer {self.ACCESS_TOKEN}",
            "appKey": self.api.APP_KEY,
            "appSecret": self.api.APP_SECRET,
            "tr_id": f"{self.tr_id_prefix}TTC8908R",  # tr_id 변경
            "custtype": "P",
        }
        params = {
            "CANO": self.CANO,
            "ACNT_PRDT_CD": self.ACNT_PRDT_CD,
            "PDNO": code,
            "ORD_UNPR": "0",
            "ORD_DVSN": "01",
            "CMA_EVLU_AMT_ICLD_YN": "Y",
            "OVRS_ICLD_YN": "Y",
        }
        time.sleep(0.05)
        # print(params, headers)
        res = requests.get(URL, headers=headers, params=params)
        buy_qty = res.json()["output"]["nrcvb_buy_qty"]
        return int(buy_qty)

    def buy_stock(self, code="005930", qty=1):
        # 주식 매수
        PATH = "uapi/domestic-stock/v1/trading/order-cash"
        URL = f"{self.URL_BASE}/{PATH}"
        data = {
            "CANO": self.CANO,
            "ACNT_PRDT_CD": self.ACNT_PRDT_CD,
            "PDNO": code,
            "ORD_DVSN": "01",
            "ORD_QTY": str(qty),
            "ORD_UNPR": "0",
        }
        headers = {
            "Content-Type": "application/json",
            "authorization": f"Bearer {self.ACCESS_TOKEN}",
            "appKey": self.api.APP_KEY,
            "appSecret": self.api.APP_SECRET,
            "tr_id": f"{self.tr_id_prefix}TTC0802U",  # tr_id 변경
            "custtype": "P",
        }
        
        time.sleep(0.05)
        res = requests.post(URL, headers=headers, data=json.dumps(data))
        # print(res.json())
        return res.json()

    def get_possible_sell_qty(self, code="005930"):
        # 모의투자에서는 매도 가능 조회 API를 호출하지 않음
        if self.tr_id_prefix == "V":  # 모의투자 환경에서는 매도 가능 조회하지 않음
            return 1

        # 운영계/개발계 환경에서 매도 가능 수량 조회
        PATH = "uapi/domestic-stock/v1/trading/inquire-psbl-order"
        URL = f"{self.URL_BASE}/{PATH}"
        headers = {
            "Content-Type": "application/json",
            "authorization": f"Bearer {self.ACCESS_TOKEN}",
            "appKey": self.api.APP_KEY,
            "appSecret": self.api.APP_SECRET,
            "tr_id": f"{self.tr_id_prefix}TTC8408R",  # 매도 가능 조회 tr_id
            "custtype": "P",
        }
        params = {
            "CANO": self.CANO,
            "ACNT_PRDT_CD": self.ACNT_PRDT_CD,
            "PDNO": code,
        }
        time.sleep(0.05)
        # print(headers,params)
        res = requests.get(URL, headers=headers, params=params)
        # print(res.json())
        sell_qty = res.json()["output"]["ord_psbl_qty"]
        return int(sell_qty)

    def sell_stock(self, code="005930", qty=1):
        # 주식 매도
        PATH = "uapi/domestic-stock/v1/trading/order-cash"
        URL = f"{self.URL_BASE}/{PATH}"
        data = {
            "CANO": self.CANO,
            "ACNT_PRDT_CD": self.ACNT_PRDT_CD,
            "PDNO": code,
            "ORD_DVSN": "01",
            "ORD_QTY": str(qty),
            "ORD_UNPR": "0",
        }
        headers = {
            "Content-Type": "application/json",
            "authorization": f"Bearer {self.ACCESS_TOKEN}",
            "appKey": self.api.APP_KEY,
            "appSecret": self.api.APP_SECRET,
            "tr_id": f"{self.tr_id_prefix}TTC0801U",  # tr_id 변경
            "custtype": "P",
        }
        time.sleep(0.05)
        res = requests.post(URL, headers=headers, data=json.dumps(data))
        return res.json()

    def read_last_value(self):
        # last_value.txt 파일 읽기
        try:
            with open('last_value.txt', 'r') as file:
                return int(file.read().strip())
        except FileNotFoundError:
            print("Error: 'last_value.txt' file not found.")
            return None
        except ValueError:
            print("Error: Invalid value in 'last_value.txt'.")
            return None

    def execute_trade_based_on_last_value(self, code="005930"):
        last_value = self.read_last_value()

        if last_value is None:
            print("매수/매도 여부를 결정할 수 없습니다.")
            return

        if last_value == 1:
            # 매수 실행
            buy_qty = self.get_possible_buy_qty(code)
            if buy_qty > 0:
                print(f"매수 가능한 수량은 {buy_qty}주입니다. 매수하시겠습니까? (y/n): ", end="")
                decision = input()
                if decision.lower() == "y":
                    buy_result = self.buy_stock(code, buy_qty)
                    print(f"매수 실행: 가능 수량 {buy_qty}주, 매수 결과: {buy_result}")
                else:
                    print("매수를 취소하였습니다.")
            else:
                print(f"{code}에 대해 매수 가능한 수량이 없습니다.")
        elif last_value == 0:
            # 매도 실행
            sell_qty = self.get_possible_sell_qty(code)
            if sell_qty > 0:
                print(f"매도 가능한 수량은 {sell_qty}주입니다. 매도하시겠습니까? (y/n): ", end="")
                decision = input()
                if decision.lower() == "y":
                    sell_result = self.sell_stock(code, sell_qty)
                    print(f"매도 실행: 가능 수량 {sell_qty}주, 매도 결과: {sell_result}")
                else:
                    print("매도를 취소하였습니다.")
            else:
                print(f"{code}에 대해 매도 가능한 수량이 없습니다.")
        else:
            print("Error: 'last_value.txt' 파일에 유효하지 않은 값이 있습니다. 값은 0 또는 1이어야 합니다.")

### **코드 실행**

* 종목(360200) 에 대해서 last_value.txt가 0이면 가능수량만큼 전체 매수, 1이면 가능수량만큼 전체 매도
* 사용자 입력을 받아 ① 환경설정(1: 운영계, 2: 개발계, 3: 모의투자) 및 ② 주문실행 전 체크를 받음

In [292]:
# 1. 운영계
if __name__ == "__main__":
    trader = StockTrader()

    # 주식 매수/매도 여부 결정 및 실행
    trader.execute_trade_based_on_last_value("360200")

1: 운영계, 2: 개발계, 3: 모의투자 계정 파일을 선택하세요:  1


기존 토큰을 사용합니다.
매수 가능한 수량은 17주입니다. 매수하시겠습니까? (y/n): 

 y


매수 실행: 가능 수량 17주, 매수 결과: {'rt_cd': '7', 'msg_cd': 'APBK0918', 'msg1': '장운영시간이 아닙니다.(아침동시호가개시(110) 주문불가시간)'}


In [294]:
# 2. 개발계
if __name__ == "__main__":
    trader = StockTrader()

    # 주식 매수/매도 여부 결정 및 실행
    trader.execute_trade_based_on_last_value("360200")

1: 운영계, 2: 개발계, 3: 모의투자 계정 파일을 선택하세요:  2


기존 토큰을 사용합니다.
매수 가능한 수량은 206653주입니다. 매수하시겠습니까? (y/n): 

 y


매수 실행: 가능 수량 206653주, 매수 결과: {'rt_cd': '7', 'msg_cd': 'APBK0919', 'msg1': '장운영일자가 주문일과 상이합니다'}


In [296]:
# 3. 모의투자
if __name__ == "__main__":
    trader = StockTrader()

    # 주식 매수/매도 여부 결정 및 실행
    trader.execute_trade_based_on_last_value("360200")

1: 운영계, 2: 개발계, 3: 모의투자 계정 파일을 선택하세요:  3


기존 토큰을 사용합니다.
매수 가능한 수량은 17607주입니다. 매수하시겠습니까? (y/n): 

 y


매수 실행: 가능 수량 17607주, 매수 결과: {'rt_cd': '1', 'msg_cd': '40570000', 'msg1': '모의투자 장시작전 입니다.'}
